In [1]:
# Parameters
dataset = "CIFAR10" # CIFAR10, CIFAR100, SVHN
v_type = "mingd" # mingd, rand
root_path = "/home/r545zhan/aaa/False-Claims-Against-Model-Ownership-Resolution/defences/cifar10/DI/files"
params_path = "/home/r545zhan/aaa/False-Claims-Against-Model-Ownership-Resolution/defences/cifar10/DI/files"
split_index = 500


In [2]:
from IPython.display import display, Markdown
v_type2disp = dict(mingd="MinGD", rand="Blind Walk")
display(Markdown(f"# {dataset} - {v_type2disp[v_type]} results"))

# CIFAR10 - MinGD results

In [3]:
import os, sys
sys.path.insert(0, params_path)

import argparse, time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from importlib import reload
from tqdm.auto import tqdm

import random

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

/home/r545zhan/.conda/envs/pytorch-gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import seaborn as sns
import pandas as pd
import os

In [5]:
from scipy.stats import combine_pvalues, ttest_ind_from_stats, ttest_ind
from functools import reduce
from scipy.stats import hmean

In [6]:
root = os.path.join(root_path,dataset)

In [7]:
names = ["teacher","teacher_adv","fine-tune","independent","ind","suspect","suspect_same_struct","suspect_same_data","suspect_adv","suspect_same_data_adv","suspect_same_struct_adv"]
trains = {}
tests = {}
for name in names:
    trains[name] = (torch.load(f"{root}/model_{name}/train_{v_type}_vulnerability_2.pt"))
    tests[name] = (torch.load(f"{root}/model_{name}/test_{v_type}_vulnerability_2.pt"))
mean_cifar = trains["teacher"].mean(dim = (0,1))
std_cifar = trains["teacher"].std(dim = (0,1))

In [8]:
for name in names:
    trains[name] = trains[name].sort(dim = 1)[0]
    tests[name] = tests[name].sort(dim = 1)[0]
for name in names:
    trains[name] = (trains[name]- mean_cifar)/std_cifar
    tests[name] = (tests[name]- mean_cifar)/std_cifar
f_num = 30
a_num=30

trains_n = {}
tests_n = {}
for name in names:
    if "ind" == name:
        trains_n[name] = trains[name].T.reshape(500,f_num)[:,:a_num]
        tests_n[name] = tests[name].T.reshape(500,f_num)[:,:a_num]
    else:
        trains_n[name] = trains[name].T.reshape(500,f_num)[:,:a_num]
        tests_n[name] = tests[name].T.reshape(500,f_num)[:,:a_num]

/tmp/ipykernel_910255/3891043543.py:17: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  trains_n[name] = trains[name].T.reshape(500,f_num)[:,:a_num]


In [9]:
n_ex = split_index
train = torch.cat((trains_n["teacher"][:250], tests_n["teacher"][:250]), dim = 0)
y = torch.cat((torch.zeros(250), torch.ones(250)), dim = 0)
train_adv = torch.cat((trains_n["teacher_adv"][:250], tests_n["teacher_adv"][:250]), dim = 0)
rand=torch.randperm(y.shape[0])
train = train[rand]
train_adv = train_adv[rand]
y = y[rand]

In [10]:
model = nn.Sequential(nn.Linear(a_num,100),nn.ReLU(),nn.Linear(100,1),nn.Tanh())
criterion = nn.CrossEntropyLoss()
optimizer =torch.optim.SGD(model.parameters(), lr=0.1)
with tqdm(range(500)) as pbar:
    for epoch in pbar:
        optimizer.zero_grad()
        inputs = train
        outputs = model(inputs)
        loss = -1 * ((2*y-1)*(outputs.squeeze(-1))).mean()
        loss.backward()
        optimizer.step()
        pbar.set_description('loss {}'.format(loss.item()))
model_adv = nn.Sequential(nn.Linear(a_num,100),nn.ReLU(),nn.Linear(100,1),nn.Tanh())
model_adv.load_state_dict(model.state_dict())
optimizer =torch.optim.SGD(model_adv.parameters(), lr=0.1)
with tqdm(range(500)) as pbar:
    for epoch in pbar:
        optimizer.zero_grad()
        inputs = train_adv
        outputs = model_adv(inputs)
        loss = -1 * ((2*y-1)*(outputs.squeeze(-1))).mean()
        loss.backward()
        optimizer.step()
        pbar.set_description('loss {}'.format(loss.item()))

loss -0.8583267331123352: 100%|██████████| 500/500 [00:00<00:00, 613.72it/s]


In [11]:
def get_p(outputs_train, outputs_test):
    pred_test = outputs_test[:,0].detach().cpu().numpy()
    pred_train = outputs_train[:,0].detach().cpu().numpy()
    tval, pval = ttest_ind(pred_test, pred_train, alternative="greater", equal_var=False)
    if pval < 0:
        raise Exception(f"p-value={pval}")
    return pval

def get_p_values(num_ex, train, test, k):
    total = train.shape[0]
    sum_p = 0
    p_values = []
    positions_list = []
    for i in range(k):
        positions = torch.randperm(total)[:num_ex]
        p_val = get_p(train[positions], test[positions])
        positions_list.append(positions)
        p_values.append(p_val)
    return p_values

def get_fischer(num_ex, train, test, k):
    p_values = get_p_values(num_ex, train, test, k)
    return combine_pvalues(p_values, method="mudholkar_george")[1]

def get_max_p_value(num_ex, train, test, k):
    p_values = get_p_values(num_ex, train, test, k)
    return max(p_values)

In [12]:
outputs_tr = {}
outputs_te = {}
for name in names:
    if 'adv' not in name:
        model.eval()
        outputs_tr[name] = model(trains_n[name])
        outputs_te[name] = model(tests_n[name])
    else:
        model_adv.eval()
        outputs_tr[name] = model_adv(trains_n[name])
        outputs_te[name] = model_adv(tests_n[name])

In [13]:
import scipy.stats as stats
def print_inference(outputs_train, outputs_test):
    m1, m2 = outputs_test[:,0].mean(), outputs_train[:,0].mean()
    pval = get_p(outputs_train, outputs_test)
    print(f"p-value = {pval} \t| Mean difference = {m1-m2}\t|test_mu = {m1},train_mu = {m2}")
    return (m1-m2).item()

In [14]:
for name in names:
    outputs_tr[name], outputs_te[name] = outputs_tr[name][250:], outputs_te[name][250:]

In [15]:
mu={}
max_mu=0
for name in names:
    print(f"{name}")
    mu[name]=print_inference(outputs_tr[name], outputs_te[name])
    if mu[name]>=max_mu:
        max_mu = mu[name]

teacher
p-value = 2.3356723084772207e-117 	| Mean difference = 1.6947860717773438	|test_mu = 0.9969280362129211,train_mu = -0.6978579759597778
teacher_adv
p-value = 7.436678426638033e-119 	| Mean difference = 1.67819344997406	|test_mu = 0.9929133653640747,train_mu = -0.6852800846099854
fine-tune
p-value = 8.564214471473425e-66 	| Mean difference = 1.243217945098877	|test_mu = 0.9971282482147217,train_mu = -0.24608971178531647
independent
p-value = 0.9901923237278946 	| Mean difference = -0.00306093692779541	|test_mu = 0.9968839287757874,train_mu = 0.9999448657035828
ind
p-value = 9.271641757929135e-85 	| Mean difference = 1.447338581085205	|test_mu = 0.9969323873519897,train_mu = -0.4504062533378601
suspect
p-value = 1.2662814785702517e-86 	| Mean difference = 1.4428024291992188	|test_mu = 0.9943685531616211,train_mu = -0.44843387603759766
suspect_same_struct
p-value = 2.2403108752476746e-94 	| Mean difference = 1.5191460847854614	|test_mu = 0.9969804883003235,train_mu = -0.52216559648

In [16]:
for name in names:
    print(name)
    print(mu[name]/max_mu)

teacher
0.9960840166616362
teacher_adv
0.9863319626130508
fine-tune
0.7306819102196431
independent
-0.0017990178232872233
ind
0.8506506227094525
suspect
0.8479845703586433
suspect_same_struct
0.8928543603394095
suspect_same_data
0.9027507449840914
suspect_adv
1.0
suspect_same_data_adv
0.9910034395522445
suspect_same_struct_adv
0.9848485103260834
